#Import files and read in as dataframes

In [1]:
# reading in files from google drive to speed up run time
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

##Crashes
https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
Crash data shows information about each traffic crash on city streets within the City of Chicago limits and under the jurisdiction of Chicago Police Department (CPD). Data are shown as is from the electronic crash reporting system (E-Crash) at CPD, excluding any personally identifiable information. Records are added to the data portal when a crash report is finalized or when amendments are made to an existing report in E-Crash. Data from E-Crash are available for some police districts in 2015, but citywide data are not available until September 2017. About half of all crash reports, mostly minor crashes, are self-reported at the police district by the driver(s) involved and the other half are recorded at the scene by the police officer responding to the crash. Many of the crash parameters, including street condition data, weather condition, and posted speed limits, are recorded by the reporting officer based on best available information at the time, but many of these may disagree with posted information or other assessments on road conditions. If any new or updated information on a crash is received, the reporting officer may amend the crash report at a later time. A traffic crash within the city limits for which CPD is not the responding police agency, typically crashes on interstate highways, freeway ramps, and on local roads along the City boundary, are excluded from this dataset.

In [3]:
crashes_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_Crashes.csv")

In [43]:
crashes_df.columns

Index(['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE',
       'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND',
       'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'INTERSECTION_RELATED_I',
       'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LA

##People
https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d
This data contains information about people involved in a crash and if any injuries were sustained. This dataset should be used in combination with the traffic Crash and Vehicle dataset. Each record corresponds to an occupant in a vehicle listed in the Crash dataset. Some people involved in a crash may not have been an occupant in a motor vehicle, but may have been a pedestrian, bicyclist, or using another non-motor vehicle mode of transportation. Injuries reported are reported by the responding police officer. Fatalities that occur after the initial reports are typically updated in these records up to 30 days after the date of the crash. Person data can be linked with the Crash and Vehicle dataset using the “CRASH_RECORD_ID” field. A vehicle can have multiple occupants and hence have a one to many relationship between Vehicle and Person dataset. However, a pedestrian is a “unit” by itself and have a one to one relationship between the Vehicle and Person table.

In [4]:
people_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_People.csv")

<ipython-input-4-8866f2488915>:1: DtypeWarning: Columns (20,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  people_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_People.csv")


In [5]:
# replacing space in column name with underscore
people_df.rename(columns={"BAC_RESULT VALUE": "BAC_RESULT_VALUE"}, inplace=True)

#Steps for creating master dataset

1. Keep only the columns we want
2. clean data and trouble shoot (match data types, consistent formatting, etc)
3. merge into one data set
4. Filter by date: 2018 - 2023 (keep in mind city budgeting season begins in June)

###Crashes dataframe edit

In [44]:
selected_columns = ['CRASH_RECORD_ID', 'CRASH_DATE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'POSTED_SPEED_LIMIT','WEATHER_CONDITION',
                    'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE', 'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE', 'MOST_SEVERE_INJURY',
                    'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'LATITUDE', 'LONGITUDE','LOCATION']
crashes_edit_df = crashes_df[selected_columns].copy()

In [45]:
# forcing datetime
crashes_edit_df['CRASH_DATE'] = pd.to_datetime(crashes_edit_df['CRASH_DATE'], errors='coerce' )

In [46]:
crashes_edit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739105 entries, 0 to 739104
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          739105 non-null  object        
 1   CRASH_DATE               739105 non-null  datetime64[ns]
 2   CRASH_HOUR               739105 non-null  int64         
 3   CRASH_DAY_OF_WEEK        739105 non-null  int64         
 4   CRASH_MONTH              739105 non-null  int64         
 5   POSTED_SPEED_LIMIT       739105 non-null  int64         
 6   WEATHER_CONDITION        739105 non-null  object        
 7   LIGHTING_CONDITION       739105 non-null  object        
 8   ROADWAY_SURFACE_COND     739105 non-null  object        
 9   ROAD_DEFECT              739105 non-null  object        
 10  CRASH_TYPE               739105 non-null  object        
 11  HIT_AND_RUN_I            230382 non-null  object        
 12  PRIM_CONTRIBUTOR

In [52]:
# filtering by date
crashes_edit_df_2018_2023 = crashes_edit_df[(crashes_edit_df['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [53]:
crashes_edit_df_2018_2023.sort_values(by='CRASH_DATE')

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,LATITUDE,LONGITUDE,LOCATION
111894,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,2018-01-01 00:00:00,0,2,1,25,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.681735,-87.641014,POINT (-87.641013916869 41.681735373094)
631253,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,2018-01-01 00:00:00,0,2,1,20,CLEAR,DAYLIGHT,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.800575,-87.589225,POINT (-87.589225075066 41.80057461981)
209356,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,2018-01-01 00:00:00,0,2,1,30,CLEAR,UNKNOWN,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.955842,-87.650268,POINT (-87.65026813462 41.955842185147)
573304,dfb3229940c672426c4010d7348d16b69b030191213b86...,2018-01-01 00:05:00,0,2,1,30,CLEAR,"DARKNESS, LIGHTED ROAD",UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.949417,-87.668774,POINT (-87.66877431033 41.949417412921)
40841,821ee3270b2c503fd0905415f674d3f4a63e1ac15d9100...,2018-01-01 00:07:00,0,2,1,35,SNOW,"DARKNESS, LIGHTED ROAD",SNOW OR SLUSH,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,WEATHER,NO INDICATION OF INJURY,0.0,0.0,0.0,41.683946,-87.622993,POINT (-87.622993129041 41.683945885086)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700084,19c606950b068ff6e704f68bcd42ac5379e3142850951e...,2023-07-06 22:20:00,22,5,7,30,CLEAR,DUSK,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,IMPROPER LANE USAGE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.880345,-87.765601,POINT (-87.765601346906 41.880344977442)
705198,a1f562015cbc6bfecec083ee97dad0cca3d9f770682764...,2023-07-06 22:45:00,22,5,7,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,IMPROPER TURNING/NO SIGNAL,NO INDICATION OF INJURY,0.0,0.0,0.0,41.875533,-87.633804,POINT (-87.633803891217 41.875532652069)
708813,16ec144e4a39707b2d7c51fd924bc53dff5894de75d085...,2023-07-07 00:17:00,0,6,7,25,CLEAR,DARKNESS,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.843576,-87.705160,POINT (-87.705160226138 41.843575982562)
708775,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,0,6,7,30,CLEAR,DAWN,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,FAILING TO YIELD RIGHT-OF-WAY,NO INDICATION OF INJURY,0.0,0.0,0.0,41.892623,-87.763330,POINT (-87.763330444094 41.892623022942)


In [54]:
crashes_edit_df_2018_2023.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601186 entries, 0 to 739104
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          601186 non-null  object        
 1   CRASH_DATE               601186 non-null  datetime64[ns]
 2   CRASH_HOUR               601186 non-null  int64         
 3   CRASH_DAY_OF_WEEK        601186 non-null  int64         
 4   CRASH_MONTH              601186 non-null  int64         
 5   POSTED_SPEED_LIMIT       601186 non-null  int64         
 6   WEATHER_CONDITION        601186 non-null  object        
 7   LIGHTING_CONDITION       601186 non-null  object        
 8   ROADWAY_SURFACE_COND     601186 non-null  object        
 9   ROAD_DEFECT              601186 non-null  object        
 10  CRASH_TYPE               601186 non-null  object        
 11  HIT_AND_RUN_I            192710 non-null  object        
 12  PRIM_CONTRIBUTOR

###People dataframe edit

In [68]:
selected_columns = ['PERSON_ID', 'CRASH_RECORD_ID', 'VEHICLE_ID', 'CRASH_DATE', 'PERSON_TYPE', 'SEX', 'AGE', 'HOSPITAL', 'INJURY_CLASSIFICATION',
                    'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT', 'BAC_RESULT_VALUE']
people_edit_df = people_df[selected_columns].copy()

In [69]:
# Create 'is_driver' column in the 'people' dataframe
people_edit_df['is_driver'] = (people_edit_df['PERSON_TYPE'] == 'DRIVER').astype(int)

# Create 'total_drivers_in_crash' column by grouping by 'CRASH_RECORD_ID' and summing 'is_driver'
people_edit_df['total_drivers_in_crash'] = people_edit_df.groupby('CRASH_RECORD_ID')['is_driver'].transform('sum')

In [70]:
# forcing datetime
people_edit_df['CRASH_DATE'] = pd.to_datetime(people_edit_df['CRASH_DATE'], errors='coerce' )

In [71]:
people_edit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620956 entries, 0 to 1620955
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   PERSON_ID               1620956 non-null  object        
 1   CRASH_RECORD_ID         1620956 non-null  object        
 2   VEHICLE_ID              1589096 non-null  float64       
 3   CRASH_DATE              1620956 non-null  datetime64[ns]
 4   PERSON_TYPE             1620956 non-null  object        
 5   SEX                     1595043 non-null  object        
 6   AGE                     1148419 non-null  float64       
 7   HOSPITAL                275410 non-null   object        
 8   INJURY_CLASSIFICATION   1620267 non-null  object        
 9   DRIVER_ACTION           1289959 non-null  object        
 10  DRIVER_VISION           1289491 non-null  object        
 11  PHYSICAL_CONDITION      1290851 non-null  object        
 12  BAC_RESULT    

In [72]:
# filtering by date
people_edit_df_2018_2023 = people_edit_df[(people_edit_df['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [73]:
people_edit_df_2018_2023.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1318659 entries, 0 to 1620955
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   PERSON_ID               1318659 non-null  object        
 1   CRASH_RECORD_ID         1318659 non-null  object        
 2   VEHICLE_ID              1291471 non-null  float64       
 3   CRASH_DATE              1318659 non-null  datetime64[ns]
 4   PERSON_TYPE             1318659 non-null  object        
 5   SEX                     1296738 non-null  object        
 6   AGE                     936778 non-null   float64       
 7   HOSPITAL                231395 non-null   object        
 8   INJURY_CLASSIFICATION   1318202 non-null  object        
 9   DRIVER_ACTION           1046639 non-null  object        
 10  DRIVER_VISION           1046214 non-null  object        
 11  PHYSICAL_CONDITION      1047428 non-null  object        
 12  BAC_RESULT    

In [20]:
people_edit_df_2018_2023.sort_values(by='CRASH_DATE')

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,PERSON_TYPE,SEX,AGE,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash
703337,O294415,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,285514.0,2018-01-01 00:00:00,DRIVER,X,NaN,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1
703406,O294495,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,285601.0,2018-01-01 00:00:00,DRIVER,X,NaN,DNA,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1
703194,O294262,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,285396.0,2018-01-01 00:00:00,DRIVER,X,NaN,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1
1314562,P64097,dfb3229940c672426c4010d7348d16b69b030191213b86...,285043.0,2018-01-01 00:05:00,PASSENGER,M,NaN,REFUSED,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,0,1
702919,O293914,dfb3229940c672426c4010d7348d16b69b030191213b86...,285043.0,2018-01-01 00:05:00,DRIVER,M,53.0,REFUSED,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558021,O1612807,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,1534732.0,2023-07-07 00:19:00,DRIVER,F,29.0,NaN,NO INDICATION OF INJURY,FAILED TO YIELD,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1,2
1558022,O1612808,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,1534737.0,2023-07-07 00:19:00,DRIVER,M,19.0,NaN,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1,2
1558638,P356740,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,1534737.0,2023-07-07 00:19:00,PASSENGER,M,22.0,NaN,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,0,2
1558024,O1612817,c449bb5ee04a25261ec126d7ff1ee42cafce8e11ed9230...,1534744.0,2023-07-07 00:30:00,DRIVER,M,NaN,NaN,NO INDICATION OF INJURY,OTHER,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1


###Cleaning up `CRASH_RECORD_ID` as we will be matching on this column

In [74]:
#stripping trailing and leading spaces on universal ID column
crashes_edit_df_2018_2023['CRASH_RECORD_ID'] = crashes_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()
people_edit_df_2018_2023['CRASH_RECORD_ID'] = people_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()


<ipython-input-74-c6c98df74d07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_edit_df_2018_2023['CRASH_RECORD_ID'] = crashes_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()
<ipython-input-74-c6c98df74d07>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_edit_df_2018_2023['CRASH_RECORD_ID'] = people_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()


###Confirming datatypes of common ID's

In [57]:
# Printing ID datatypes
crashes_crash_record_id = crashes_edit_df['CRASH_RECORD_ID'].dtype
people_crash_record_id = people_edit_df['CRASH_RECORD_ID'].dtype




#Crashes
print(f"In crashes, crash record id data type= {crashes_crash_record_id}")

#People
print(f"In people, crash record id data type={people_crash_record_id}")





In crashes, crash record id data type= object
In people, crash record id data type=object


#Organizing and renaming columns

*   a suffix of _c indicates a column origin of `crashes.csv`
*   a suffix of _p indicates a column origin of `people.csv`



In [75]:
# get list of columns
people_edit_df_2018_2023.columns

Index(['PERSON_ID', 'CRASH_RECORD_ID', 'VEHICLE_ID', 'CRASH_DATE',
       'PERSON_TYPE', 'SEX', 'AGE', 'HOSPITAL', 'INJURY_CLASSIFICATION',
       'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT',
       'BAC_RESULT_VALUE', 'is_driver', 'total_drivers_in_crash'],
      dtype='object')

In [26]:
# adding _p to all columns except 'CRASH_RECORD_ID' as we need to match on that
people_edit_df_2018_2023.rename(columns={"PERSON_ID": "PERSON_ID_p", "VEHICLE_ID":"VEHICLE_ID_p", "CRASH_DATE": "CRASH_DATE_p", "PERSON_TYPE":"PERSON_TYPE_p",
                                         'SEX':'SEX_p', 'AGE':'AGE_p', 'HOSPITAL':'HOSPITAL_p', 'INJURY_CLASSIFICATION':'INJURY_CLASSIFICATION_p',
                                        'DRIVER_ACTION':'DRIVER_ACTION_p', 'DRIVER_VISION':'DRIVER_VISION_p', 'PHYSICAL_CONDITION':'PHYSICAL_CONDITION_p',
                                         'BAC_RESULT':'BAC_RESULT_p','BAC_RESULT_VALUE':'BAC_RESULT_VALUE_p', 'is_driver':'is_driver_p', 'total_drivers_in_crash':'total_drivers_in_crash_p'}, inplace=True)

<ipython-input-26-0e3652616659>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_edit_df_2018_2023.rename(columns={"PERSON_ID": "PERSON_ID_p", "CRASH_RECORD_ID": "CRASH_RECORD_ID_p","VEHICLE_ID":"VEHICLE_ID_p", "CRASH_DATE": "CRASH_DATE_p", "PERSON_TYPE":"PERSON_TYPE_p",


In [39]:
# confirming updated column names
people_edit_df_2018_2023.columns

Index(['PERSON_ID_p', 'CRASH_RECORD_ID', 'VEHICLE_ID_p', 'CRASH_DATE_p',
       'PERSON_TYPE_p', 'SEX_p', 'AGE_p', 'HOSPITAL_p',
       'INJURY_CLASSIFICATION_p', 'DRIVER_ACTION_p', 'DRIVER_VISION_p',
       'PHYSICAL_CONDITION_p', 'BAC_RESULT_p', 'BAC_RESULT_VALUE_p',
       'is_driver_p', 'total_drivers_in_crash_p'],
      dtype='object')

In [59]:
# get list of column names
crashes_edit_df_2018_2023.columns

Index(['CRASH_RECORD_ID', 'CRASH_DATE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'POSTED_SPEED_LIMIT', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'LATITUDE', 'LONGITUDE', 'LOCATION'],
      dtype='object')

In [60]:
# adding _c to all columns except 'CRASH_RECORD_ID' as we need to match on that
crashes_edit_df_2018_2023.rename(columns={'CRASH_DATE':'CRASH_DATE_c', 'CRASH_HOUR':'CRASH_HOUR_c',
                                          'CRASH_DAY_OF_WEEK':'CRASH_DAY_OF_WEEK_c','CRASH_MONTH':'CRASH_MONTH_c', 'POSTED_SPEED_LIMIT':'POSTED_SPEED_LIMIT_c',
                                          'WEATHER_CONDITION':'WEATHER_CONDITION_c','LIGHTING_CONDITION': 'LIGHTING_CONDITION_c', 'ROADWAY_SURFACE_COND':'ROADWAY_SURFACE_COND_c',
                                          'ROAD_DEFECT':'ROAD_DEFECT_c', 'CRASH_TYPE':'CRASH_TYPE_c','HIT_AND_RUN_I':'HIT_AND_RUN_I_c', 'PRIM_CONTRIBUTORY_CAUSE':'PRIM_CONTRIBUTORY_CAUSE_c',
                                          'MOST_SEVERE_INJURY':'MOST_SEVERE_INJURY_c', 'INJURIES_TOTAL':'INJURIES_TOTAL_c', 'INJURIES_FATAL':'INJURIES_FATAL_c','INJURIES_INCAPACITATING':'INJURIES_INCAPACITATING_c', 'LATITUDE': 'LATITUDE_c', 'LONGITUDE':'LONGITUDE_c','LOCATION':'LOCATION_c'}, inplace=True)

<ipython-input-60-c07ce5c49673>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_edit_df_2018_2023.rename(columns={'CRASH_DATE':'CRASH_DATE_c', 'CRASH_HOUR':'CRASH_HOUR_c',


In [61]:
# confirming updated column names
crashes_edit_df_2018_2023.columns

Index(['CRASH_RECORD_ID', 'CRASH_DATE_c', 'CRASH_HOUR_c',
       'CRASH_DAY_OF_WEEK_c', 'CRASH_MONTH_c', 'POSTED_SPEED_LIMIT_c',
       'WEATHER_CONDITION_c', 'LIGHTING_CONDITION_c', 'ROADWAY_SURFACE_COND_c',
       'ROAD_DEFECT_c', 'CRASH_TYPE_c', 'HIT_AND_RUN_I_c',
       'PRIM_CONTRIBUTORY_CAUSE_c', 'MOST_SEVERE_INJURY_c', 'INJURIES_TOTAL_c',
       'INJURIES_FATAL_c', 'INJURIES_INCAPACITATING_c', 'LATITUDE_c',
       'LONGITUDE_c', 'LOCATION_c'],
      dtype='object')

#Merging `people` with `crashes`


In [62]:
# doing a left join with people on the left as we want to stucture our data around individuals involved in car crashes
merged_df = pd.merge(people_edit_df_2018_2023, crashes_edit_df_2018_2023, on='CRASH_RECORD_ID', how='left')

In [63]:
# previewing the first 5 records of our new dataframe
merged_df.head()

,PERSON_ID_p,CRASH_RECORD_ID,VEHICLE_ID_p,CRASH_DATE_p,PERSON_TYPE_p,SEX_p,AGE_p,HOSPITAL_p,INJURY_CLASSIFICATION_p,DRIVER_ACTION_p,...,CRASH_TYPE_c,HIT_AND_RUN_I_c,PRIM_CONTRIBUTORY_CAUSE_c,MOST_SEVERE_INJURY_c,INJURIES_TOTAL_c,INJURIES_FATAL_c,INJURIES_INCAPACITATING_c,LATITUDE_c,LONGITUDE_c,LOCATION_c
0,O749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,2019-09-28 03:30:00,DRIVER,M,25.0,NaN,NO INDICATION OF INJURY,UNKNOWN,...,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.900043,-87.755577,POINT (-87.755576950444 41.900042872883)
1,O871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,2020-04-13 22:50:00,DRIVER,M,37.0,NaN,NO INDICATION OF INJURY,NONE,...,NO INJURY / DRIVE AWAY,Y,IMPROPER OVERTAKING/PASSING,NO INDICATION OF INJURY,0.0,0.0,0.0,41.736044,-87.653404,POINT (-87.653404241798 41.736044089544)
2,O848601,f25f09798b51603bde602ded37fea826fc8b7f962fc3b0...,805348.0,2020-02-23 18:15:00,DRIVER,F,34.0,NaN,NO INDICATION OF INJURY,UNKNOWN,...,NO INJURY / DRIVE AWAY,NaN,IMPROPER OVERTAKING/PASSING,NO INDICATION OF INJURY,0.0,0.0,0.0,41.997662,-87.700128,POINT (-87.700128203434 41.99766204922)
3,O877654,e9146986f4b0884d00ff3a54da5249263b4b36c15d01ce...,832624.0,2020-04-30 15:05:00,DRIVER,M,53.0,REFUSED,NO INDICATION OF INJURY,UNKNOWN,...,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.851561,-87.710734,POINT (-87.710733928594 41.851560864391)
4,O879085,f15ccbd94a8e29ce8424882ce93061d4e1d0deb214acfe...,833984.0,2020-05-03 22:30:00,DRIVER,M,65.0,NaN,NO INDICATION OF INJURY,UNKNOWN,...,NO INJURY / DRIVE AWAY,N,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,41.699089,-87.559695,POINT (-87.559694671759 41.699088717329)


In [64]:
# checking for non-null counts and data types

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1318659 entries, 0 to 1318658
Data columns (total 35 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   PERSON_ID_p                1318659 non-null  object        
 1   CRASH_RECORD_ID            1318659 non-null  object        
 2   VEHICLE_ID_p               1291471 non-null  float64       
 3   CRASH_DATE_p               1318659 non-null  datetime64[ns]
 4   PERSON_TYPE_p              1318659 non-null  object        
 5   SEX_p                      1296738 non-null  object        
 6   AGE_p                      936778 non-null   float64       
 7   HOSPITAL_p                 231395 non-null   object        
 8   INJURY_CLASSIFICATION_p    1318202 non-null  object        
 9   DRIVER_ACTION_p            1046639 non-null  object        
 10  DRIVER_VISION_p            1046214 non-null  object        
 11  PHYSICAL_CONDITION_p       1047428 no

In [41]:
# compare to values from our filtered people dataframe
people_edit_df_2018_2023.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1318659 entries, 0 to 1620955
Data columns (total 16 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   PERSON_ID_p               1318659 non-null  object        
 1   CRASH_RECORD_ID           1318659 non-null  object        
 2   VEHICLE_ID_p              1291471 non-null  float64       
 3   CRASH_DATE_p              1318659 non-null  datetime64[ns]
 4   PERSON_TYPE_p             1318659 non-null  object        
 5   SEX_p                     1296738 non-null  object        
 6   AGE_p                     936778 non-null   float64       
 7   HOSPITAL_p                231395 non-null   object        
 8   INJURY_CLASSIFICATION_p   1318202 non-null  object        
 9   DRIVER_ACTION_p           1046639 non-null  object        
 10  DRIVER_VISION_p           1046214 non-null  object        
 11  PHYSICAL_CONDITION_p      1047428 non-null  object

In [65]:
# compare to values from our filtered crashes dataframe
crashes_edit_df_2018_2023.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601186 entries, 0 to 739104
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   CRASH_RECORD_ID            601186 non-null  object        
 1   CRASH_DATE_c               601186 non-null  datetime64[ns]
 2   CRASH_HOUR_c               601186 non-null  int64         
 3   CRASH_DAY_OF_WEEK_c        601186 non-null  int64         
 4   CRASH_MONTH_c              601186 non-null  int64         
 5   POSTED_SPEED_LIMIT_c       601186 non-null  int64         
 6   WEATHER_CONDITION_c        601186 non-null  object        
 7   LIGHTING_CONDITION_c       601186 non-null  object        
 8   ROADWAY_SURFACE_COND_c     601186 non-null  object        
 9   ROAD_DEFECT_c              601186 non-null  object        
 10  CRASH_TYPE_c               601186 non-null  object        
 11  HIT_AND_RUN_I_c            192710 non-null  object  

In [66]:
merged_df.columns

Index(['PERSON_ID_p', 'CRASH_RECORD_ID', 'VEHICLE_ID_p', 'CRASH_DATE_p',
       'PERSON_TYPE_p', 'SEX_p', 'AGE_p', 'HOSPITAL_p',
       'INJURY_CLASSIFICATION_p', 'DRIVER_ACTION_p', 'DRIVER_VISION_p',
       'PHYSICAL_CONDITION_p', 'BAC_RESULT_p', 'BAC_RESULT_VALUE_p',
       'is_driver_p', 'total_drivers_in_crash_p', 'CRASH_DATE_c',
       'CRASH_HOUR_c', 'CRASH_DAY_OF_WEEK_c', 'CRASH_MONTH_c',
       'POSTED_SPEED_LIMIT_c', 'WEATHER_CONDITION_c', 'LIGHTING_CONDITION_c',
       'ROADWAY_SURFACE_COND_c', 'ROAD_DEFECT_c', 'CRASH_TYPE_c',
       'HIT_AND_RUN_I_c', 'PRIM_CONTRIBUTORY_CAUSE_c', 'MOST_SEVERE_INJURY_c',
       'INJURIES_TOTAL_c', 'INJURIES_FATAL_c', 'INJURIES_INCAPACITATING_c',
       'LATITUDE_c', 'LONGITUDE_c', 'LOCATION_c'],
      dtype='object')

In [67]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/merged_df.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  merged_df.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
